<a href="https://colab.research.google.com/github/guillem-ms/IA-RM/blob/main/Detectron2vsYOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detectron2 Vs YOLOv5

After training both models, we test and compare them.

<a target="_blank"  href="https://colab.research.google.com/drive/15xa6kVtLJNeWKDKmEN0pF5YLm6h8yJ9C#scrollTo=wPvaEIlxq_Bd">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 276kB 14.0MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=d0621d9b796288ab8ef3ade51c9f6672ba5fc003460ce5881b0e3ff1a6e6a3d4
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.8.1+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.2MB 823kB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210317-cp37-none-any.whl size=58543 sha256=5c3f3f8c85ad81a6d38d965dfc4a7560c238c51de59b3e1c2d12059858cb9ed4
  Stored in directory: /root/.cache/pip/wheels/d2/ee/3a/5c531df777c03d8c67f22c65f97d6f75321087482d05a9b218
Successfully built fvcore


In [ ]:
#Mount drive so we can upload final_models

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!unzip /content/drive/MyDrive/Models/model_final.zip

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data.datasets import register_coco_instances

register_coco_instances("simObjects_test", {}, "data/test.json", "data/test")

AssertionError: ignored

In [ ]:
import random
from detectron2.data import DatasetCatalog, MetadataCatalog
from google.colab.patches import cv2_imshow

dataset_dicts = DatasetCatalog.get("simObjects_test")
simObjects_train_metadata = MetadataCatalog.get("simObjects_test")

WARNING [05/30 08:35:12 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/30 08:35:12 d2.data.datasets.coco]: Loaded 40 images in COCO format from data/test.json


In [ ]:
from detectron2.config import get_cfg
import os

cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = "model_final.pth"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("simObjects_test", ("bbox", "segm"), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "simObjects_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

WARNING [05/30 08:35:25 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/30 08:35:25 d2.data.datasets.coco]: Loaded 40 images in COCO format from data/test.json
[05/30 08:35:25 d2.data.build]: Distribution of instances among all 9 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
|     Apple     | 8            |    Banana     | 16           | BluePillsGl.. | 8            |
| BluePurpleP.. | 7            |     Glass     | 8            | GreenBlackP.. | 13           |
|    Orange     | 10           | RedPillsGlass | 13           |  TV Command   | 10           |
|               |              |               |              |               |              |
|     total     | 93           |               |              |               |              |
[05/30 08:35:25 d2.data

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[05/30 08:35:29 d2.evaluation.evaluator]: Inference done 11/40. 0.2712 s / img. ETA=0:00:07
[05/30 08:35:34 d2.evaluation.evaluator]: Inference done 30/40. 0.2685 s / img. ETA=0:00:02
[05/30 08:35:37 d2.evaluation.evaluator]: Total inference time: 0:00:09.683517 (0.276672 s / img per device, on 1 devices)
[05/30 08:35:37 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:09 (0.270022 s / img per device, on 1 devices)
[05/30 08:35:37 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/30 08:35:37 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[05/30 08:35:37 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[05/30 08:35:37 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[05/30 08:35:37 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.01 seconds.
[05/30 08:35:37 d2.ev

# YOLOv5

In [ ]:
%%capture
!git clone https://github.com/ultralytics/yolov5.git
!curl -L "https://app.roboflow.com/ds/o032xnyOfQ?key=F1k7Kq0q2j" > roboflow.zip; unzip -o roboflow.zip; rm roboflow.zip
%cd yolov5
!pip install -r requirements.txt

In [ ]:
%%capture
!unzip /content/drive/MyDrive/Models/YOLOv5.zip

In [ ]:
# Run YOLOv5x on
!python test.py --weights best.pt --data data.yaml --img 512 --iou 0.65 --task test

Namespace(augment=False, batch_size=32, conf_thres=0.001, data='data.yaml', device='', exist_ok=False, img_size=512, iou_thres=0.65, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='test', verbose=False, weights=['best.pt'])
YOLOv5 🚀 v5.0-128-g4b52e19 torch 1.8.1+cu101 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7075486 parameters, 0 gradients, 16.4 GFLOPS
test: Scanning '../test/labels' images and labels... 18 found, 0 missing, 0 empty, 0 corrupted: 100% 18/18 [00:00<00:00, 1362.92it/s]
test: New cache created: ../test/labels.cache
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100% 1/1 [00:00<00:00,  1.08it/s]
                 all          18          42       0.831       0.945       0.876       0.762
               Apple          18           5       0.811           1       0.929       0.819
              Banana          18   

In [ ]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/best.pt')
model.conf = 0.6  # confidence threshold (0-1)
# Image
img = '/content/test/images/img_apple_2_png.rf.519fd063980180d43e36d1fcd5060ae4.jpg'

imgreal = cv2.imread('/content/test/images/img_apple_2_png.rf.519fd063980180d43e36d1fcd5060ae4.jpg')
# Inference
results = model(imgreal)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


Fusing layers... 



requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Model Summary: 224 layers, 7075486 parameters, 0 gradients, 16.4 GFLOPS
Adding AutoShape... 
YOLOv5 🚀 2021-5-30 torch 1.8.1+cu101 CUDA:0 (Tesla K80, 11441.1875MB)



In [ ]:
results.save()

Saved image0.jpg to runs/hub/exp


In [ ]:
results.xyxy # img1 predictions (tensor)

[tensor([[226.68748, 306.96249, 310.21249, 392.27499,   0.75488,   3.00000]], device='cuda:0')]